In [ ]:
!pip install tfkit==0.7.21 nlp2go==0.4.12
!pip install git+https://github.com/roemmele/answerquest.git
!pip install torch==1.10.0+cu111 -f https://download.pytorch.org/whl/cu111/torch_stable.html

     |████████████████████████████████| 78 kB 2.8 MB/s 
     |████████████████████████████████| 1.2 MB 9.0 MB/s 
     |████████████████████████████████| 125 kB 55.3 MB/s 
     |████████████████████████████████| 4.0 MB 47.7 MB/s 
     |████████████████████████████████| 5.8 MB 29.0 MB/s 
     |████████████████████████████████| 77 kB 6.6 MB/s 
     |████████████████████████████████| 895 kB 59.8 MB/s 
     |████████████████████████████████| 596 kB 57.7 MB/s 
     |████████████████████████████████| 6.5 MB 41.2 MB/s 
     |████████████████████████████████| 251 kB 73.6 MB/s 
     |████████████████████████████████| 58 kB 4.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Cloning https://github.com/roemmele/answerquest.git to /tmp/pip-req-build-5qqn2s30
  Running command git clone -q https://github.com/roemmele/answerquest.git /tmp/pip-req-build-5qqn2s30
     |██████████████████████████

Looking in links: https://download.pytorch.org/whl/cu111/torch_stable.html
     |████████████▌                   | 834.1 MB 1.6 MB/s eta 0:13:14tcmalloc: large alloc 1147494400 bytes == 0x55c6c3bba000 @  0x7f5d14fff615 0x55c6c026317c 0x55c6c034347a 0x55c6c0265f9d 0x55c6c0357d4d 0x55c6c02d9ec8 0x55c6c02d4a2e 0x55c6c026788a 0x55c6c02d9d30 0x55c6c02d4a2e 0x55c6c026788a 0x55c6c02d6719 0x55c6c0358b76 0x55c6c02d5d95 0x55c6c0358b76 0x55c6c02d5d95 0x55c6c0358b76 0x55c6c02d5d95 0x55c6c0267ce9 0x55c6c02ab579 0x55c6c0266902 0x55c6c02d9c4d 0x55c6c02d4a2e 0x55c6c026788a 0x55c6c02d6719 0x55c6c02d4a2e 0x55c6c026788a 0x55c6c02d58f6 0x55c6c02677aa 0x55c6c02d5b4f 0x55c6c02d4a2e
     |███████████████▉                | 1055.7 MB 1.4 MB/s eta 0:12:58tcmalloc: large alloc 1434370048 bytes == 0x55c708210000 @  0x7f5d14fff615 0x55c6c026317c 0x55c6c034347a 0x55c6c0265f9d 0x55c6c0357d4d 0x55c6c02d9ec8 0x55c6c02d4a2e 0x55c6c026788a 0x55c6c02d9d30 0x55c6c02d4a2e 0x55c6c026788a 0x55c6c02d6719 0x55c6c0358b76 0x55c6

In [ ]:
# import answerquest 
from answerquest import QnAPipeline
import nlp2go

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def initialise(qg_tokenizer_path, qg_model_path, qa_model_path, BDG_model_path ):
  qna_pipeline = QnAPipeline(
    qg_tokenizer_path=qg_tokenizer_path,
    qg_model_path=qg_model_path,
    qa_model_path=qa_model_path
    # qg_tokenizer_path="qg_augmented_model/gpt2_tokenizer_vocab/",
    # qg_model_path="qg_augmented_model/qg_augmented_model.pt",
    # qa_model_path="doc_qa_model/checkpoint-59499/"
  )

  bdg_pm_model = nlp2go.Model(BDG_model_path)
  # bdg_pm_model = nlp2go.Model('./BDG_PM.pt')

  return qna_pipeline, bdg_pm_model

In [ ]:
qna_pipeline, bdg_pm_model = initialise("/content/drive/MyDrive/Sem2/IR/project/qg_augmented_model/gpt2_tokenizer_vocab/", "/content/drive/MyDrive/Sem2/IR/project/qg_augmented_model/qg_augmented_model.pt", "/content/drive/MyDrive/Sem2/IR/project/checkpoint-59499/", '/content/drive/MyDrive/Sem2/IR/project/BDG_PM.pt')

===model info===
model_config bert-base-cased
tags ['onebyone-pos_0']
type ['onebyone-pos']
maxlen 512
epoch 6
loading saved model
Using device: cpu
finish loading
loaded model predict_parameter {}


In [ ]:
def func(input_text):
  
  (sent_idxs,questions,answers) = qna_pipeline.generate_qna_items(
      input_text,
      filter_duplicate_answers=True,
      filter_redundant=True,
      sort_by_sent_order=True)

  # sent_idxs, questions, answers
  distractors = []

  result = []

  for a,b,c in zip(sent_idxs, questions, answers):

    print(a)

    strI = input_text + " [SEP] " + b + " [SEP] " + c
    input_json = {
      "input": strI
    }

    x=bdg_pm_model.predict(input_json)
    new_entry = dict()
    new_entry['question']=b
    new_entry['line_no']=a
    new_entry['ans']=c
    new_entry['distractor']=x['result'][0]
    result.append(new_entry)

  return result
  # return sent_idxs, questions, answers, distractors

In [ ]:
result = func("Mike and Morris lived in the same village. While Morris owned the largest jewelry shop in the village, Mike was a poor farmer. Both had large families with many sons, daughters-in-law and grandchildren.")
result

PRED AVG SCORE: -0.7155, PRED PPL: 2.0453
0
1
1
2


[{'ans': 'the same village',
  'distractor': 'the same village',
  'line_no': 0,
  'question': 'Where did Mike and Morris live?'},
 {'ans': 'Mike',
  'distractor': 'Jack and His wife',
  'line_no': 1,
  'question': 'Who was a poor farmer?'},
 {'ans': 'Mike and Morris',
  'distractor': 'Both and his wife',
  'line_no': 1,
  'question': 'Who owned the largest jewelry shop in the village?'},
 {'ans': 'sons, daughters-in-law and grandchildren',
  'distractor': 'Mike and daughter and daughter',
  'line_no': 2,
  'question': 'Who were the large families with?'}]

In [ ]:
# input_text = r"ONE DOLLAR AND EIGHTY-SEVEN CENTS. That was all. And sixty cents of it was in pennies. Pennies saved one and two at a time by bulldozing the grocer and the vegetable man and the butcher until one's cheek burned with the silent imputation of parsimony that such close dealing implied. Three times Della counted it. One dollar and eighty-seven cents. And the next day would be Christmas. There was clearly nothing left to do but flop down on the shabby little couch and howl. So Della did it. Which instigates the moral reflection that life is made up of sobs, sniffles, and smiles, with sniffles predominating. While the mistress of the home is gradually subsiding from the first stage to the second, take a look at the home. A furnished flat at $8 per week. It did not exactly beggar description, but it certainly had that word on the look-out for the mendicancy squad. In the vestibule below was a letter-box into which no letter would go, and an electric button from which no mortal finger could coax a ring. Also appertaining thereunto was a card bearing the name 'Mr. James Dillingham Young.' The 'Dillingham' had been flung to the breeze during a former period of prosperity when its possessor was being paid $30 per week. Now, when the income was shrunk to $20, the letters of 'Dillingham' looked blurred, as though they were thinking seriously of contracting to a modest and unassuming D. But whenever Mr. James Dillingham Young came home and reached his flat above he was called 'Jim' and greatly hugged by Mrs. James Dillingham Young, already introduced to you as Della. Which is all very good. Delia finished her cry and attended to her cheeks with the powder rag. She stood by the window and looked out dully at a grey cat walking a grey fence in a grey backyard. To-morrow would be Christmas Day, and she had only $1.87 with which to buy Jim a present. She had been saving every penny she could formonths, with this result. Twenty dollars a week doesn't go far. Expenses had been greater than she had calculated. They always are. Only $1.87 to buy a present for Jim. Her Jim. Many a happy hour she had spent planning for something nice for him. Something fine and rare and sterling - something just a little bit near to being worthy of the honour of being owned by Jim. There was a pier-glass between the windows of the room. Perhaps you have seen a pier-glass in an $8 flat. A very thin and very agile person may, by observing his reflection in a rapid sequence of longitudinal strips, obtain a fairly accurate conception of his looks. Della, being slender, had mastered the art. Suddenly she whirled from the window and stood before the glass. Her eyes were shining brilliantly, but her face had lost its colour within twenty seconds. Rapidly she pulled down her hair and let it fall to its full length. Now, there were two possessions of the James Dillingham Youngs in which they both took a mighty pride. One was Jim's gold watch that had been his father's and his grandfather's. The other was Della's hair. Had the Queen of Sheba lived in the flat across the airshaft, Della would have let her hair hang out the window some day to dry just to depreciate Her Majesty's jewels and gifts. Had King Solomon been the janitor, with all his treasures piled up in the basement, Jim would have pulled out his watch every time he passed, just to see him pluck at his beard from envy. So now Della's beautiful hair fell about her, rippling and shining like a cascade of brown waters. It reached below her knee and made itself almost a garment for her. And then she did it up again nervously and quickly. Once she faltered for a minute and stood still while a tear or two splashed on the worn red carpet. On went her old brown jacket; on went her old brown hat. With a whirl of skirts and with the brilliant sparkle still in her eyes, she fluttered out of the door and down the stairs to the street. Where she stopped the sign read: 'Mme. Sofronie. Hair Goods of All Kinds.' One flight up Della ran, and collected herself, panting. Madame, large, too white, chilly, hardly looked the 'Sofronie.' 'Will you buy my hair?' asked Della. 'I buy hair,' said Madame. 'Take yer hat off and let's have a sight at the looks of it.' Down rippled the brown cascade. 'Twenty dollars,' said Madame, lifting the mass with a practised hand."
# input_text = "Since these maintenance processes are needed to prevent damage and break-down, energy is needed for them. This energy comes from outside the body of the individual organism. So there must be a process to transfer a source of energy from outside the body of the organism, which we call food, to the inside, a process we commonly call nutrition. If the body size of the organisms is to grow, additional raw material will also be needed from outside. Since life on earth depends on carbonbased molecules, most of these food sources are also carbon-based. Depending on the complexity of these carbon sources, different organisms can then use different kinds of nutritional processes"
# input_text = "When you ' re having a holiday , one of the main questions to ask is which hotel or apartment to choose . However , when it comes to France , you have another special choice : treehouses . In France , treehouses are offered to travelers as a new choice in many places . The price may be a little higher , but you do have a chance to _ your childhood memories . Alain Laurens , one of France ' s top treehouse designers , said , \" Most of the people might have the experience of building a den when they were young . And they like that feeling of freedom when they are children . \" Its fairy - tale style gives travelers a special feeling . It seems as if they are living as a forest king and enjoying the fresh air in the morning . Another kind of treehouse is the \" star cube \" . It gives travelers the chance of looking at the stars shining in the sky when they are going to sleep . Each \" star cube \" not only offers all the comfortable things that a hotel provides for travelers , but also gives them a chance to look for stars by using a telescope . The glass roof allows you to look at the stars from your bed ."